In [1]:
from asdad.porn_creator import *

In [2]:
params = {
    # include forehead triangle transplantation
    # caution about fucking hairstyles
    'USE_FOREHEAD': False,
    # include brows transplantation
    'USE_BROWS': False,
    # if 0, output shakalization processis not applied
    # otherwise it's result img height
    # width will be adjusted proportionally
    'SHAKALIZE_HEIGHT': 0,
    # feather on the places where we cut off the eblo
    # must be odd
    'FEATHER_AMOUNT': 9,
    # thing like colour transparency 
    # 1.0 to turn off transparency and colour adjustment
    # 0..1
    'COLOUR_CORRECT_BLUR_FRAC': 0.6,
    # input face image will be less sharp
    # 1 to turn off
    # better to be int square
    'EBLO_KERNEL_FACTOR': 6,
    # the same thing bout body
    # better to be int square
    'TELKA_KERNEL_FACTOR': 2
}

im1 = cv2.imread('./asdad/pidr.jpg', cv2.IMREAD_COLOR)
im2 = cv2.imread('./asdad/porn.jpg', cv2.IMREAD_COLOR)
res, status = get_porn(im2, im1, params)
print status
cv2.imwrite('./asdad/res.jpg', res)

OK


True

In [ ]:
im1, landmarks1 = read_im_and_landmarks('./porn2.jpg')

kernel = np.ones((KERNEL_FACTOR, KERNEL_FACTOR),np.float32)/(KERNEL_FACTOR**2)
im1 = cv2.filter2D(im1, -1, kernel)
    
im2, landmarks2 = read_im_and_landmarks('./pidr.jpg')

landmarks1 = add_forehead_point(landmarks1).astype(int)
landmarks2 = add_forehead_point(landmarks2).astype(int)


M = transformation_from_points(landmarks1[ALIGN_POINTS],
                               landmarks2[ALIGN_POINTS])

mask = get_face_mask(im2, landmarks2)
warped_mask = warp_im(mask, M, im1.shape)
combined_mask = numpy.max([get_face_mask(im1, landmarks1), warped_mask],
                          axis=0)

warped_im2 = warp_im(im2, M, im1.shape)
warped_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)

output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask
output_im = resize_im(output_im, 256)

cv2.imwrite('output.jpg', output_im)

In [ ]:
#[[26,17, 27]]: left brow, right brow, upper nose (between eyes)

corner_points = [24, 19, 68]
corner_points = -landmarks1[:,0][corner_points], -landmarks1[:,1][corner_points]

# OVERLAY_POINTS = [
#     LEFT_EYE_POINTS + RIGHT_EYE_POINTS + #(LEFT_BROW_POINTS+[68]) 
#     #+ (RIGHT_BROW_POINTS+[68]),
#     (NOSE_POINTS) + MOUTH_POINTS + 
#     [24, 19, 68]
# ]


plt.scatter(-landmarks1[:,0], -landmarks1[:,1], alpha=0.5)
plt.scatter(corner_points[0], corner_points[1], color='red')

In [ ]:
# plt.scatter(-np.where(mask>0.)[1], -np.where(mask>0.)[0])
# plt.show()

im = numpy.zeros(im1.shape[:2], dtype=numpy.float64)

for group in OVERLAY_POINTS:

    points = landmarks1[group]
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color=1)
    
group = [[24, 19, 68]]

points = landmarks1[group]
points = cv2.convexHull(points)
cv2.fillConvexPoly(im, points, color=2)
    
    
im = numpy.array([im, im, im]).transpose((1, 2, 0))
im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

plt.imshow(im)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(annotate_landmarks(im1, landmarks1))
plt.imshow(warped_mask, alpha=0.5)

In [ ]:
# display(Image(filename='ava.jpg', width=200, height=100), )
# display(Image(filename='porn.jpg', width=200, height=100), )
# display(Image(filename='output.jpg', width=200, height=100), ) 

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(output_im)
# plt.imshow(warped_mask, alpha=0.5)